## Create CNN Model and Optimize it using Keras Tuner

In [4]:
!pip install keras-tuner

     |████████████████████████████████| 71kB 8.7MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp37-none-any.whl size=78938 sha256=c3484e85a13520a0d14ed6498cb0faa65c674c82c23220d487fa80eeb35b8bf7
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp37-none-any.whl size=15356 sha256=801fe27e0785daaccb6054ed7bb0363f62d5afc08113f7ad0af001147e2e7998
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [5]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [6]:
print(tf.__version__)

2.4.1


In [7]:
fashion_mnist=keras.datasets.fashion_mnist

In [8]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [9]:
train_images=train_images/255.0
test_images=test_images/255.0

In [10]:
train_images[0].shape

(28, 28)

In [11]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [12]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [13]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [14]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [15]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 00m 26s]
val_accuracy: 0.8451666831970215

Best val_accuracy So Far: 0.9148333072662354
Total elapsed time: 00h 02m 28s
INFO:tensorflow:Oracle triggered exit


In [19]:
model=tuner_search.get_best_models(num_models=1)[0]

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 48)        480       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 48)        57648     
_________________________________________________________________
flatten (Flatten)            (None, 23232)             0         
_________________________________________________________________
dense (Dense)                (None, 48)                1115184   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                490       
Total params: 1,173,802
Trainable params: 1,173,802
Non-trainable params: 0
_________________________________________________________________


In [20]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.1492 - accuracy: 0.9445 - val_loss: 0.2577 - val_accuracy: 0.9103
Epoch 5/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.1146 - accuracy: 0.9566 - val_loss: 0.2676 - val_accuracy: 0.9125
Epoch 6/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0800 - accuracy: 0.9710 - val_loss: 0.3102 - val_accuracy: 0.9172
Epoch 7/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0584 - accuracy: 0.9788 - val_loss: 0.3425 - val_accuracy: 0.9147
Epoch 8/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0453 - accuracy: 0.9832 - val_loss: 0.4055 - val_accuracy: 0.9155
Epoch 9/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0341 - accuracy: 0.9877 - val_loss: 0.3928 - val_accuracy: 0.9132
Epoch 10/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0267 - accuracy: 0.9902 - val_loss: 0.4743 - val_accuracy